In [1]:
from pandas import read_csv,DataFrame,read_excel, merge,concat, Series
import os
import re
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import gaussian_kde
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1 import make_axes_locatable

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


In [2]:
datacsv          = "peru_ap17/"
data_peru        = "country_data/"
list_csv         = os.listdir(datacsv)

In [3]:
baseline = read_csv(datacsv+"OD_baseline_Peru.csv")
baseline.index = baseline.Name
baseline["O"] = baseline.Name.apply(lambda s: re.search("(.*) - (.*)",s).group(1))
baseline["D"] = baseline.Name.apply(lambda s: re.search("(.*) - (.*)",s).group(1))

# Initial analysis: "naive" calculation of costs

In [9]:
if os.path.exists("costs_peru_ap22.csv"):
    costs_all = read_csv("costs_peru_ap22.csv")
else:
    costs_all = DataFrame(columns=["scenarioID","diff ruc from baseline (%)","diff ruc from baseline (usd)",\
                               "missingroutes",\
                               "number of affected routes","av cost with traffic (usd)",\
                               "max cost with traffic (%)","max cost with traffic (usd)",\
                               "av km difference","max km difference","tot km difference"])
    for file in list_csv:
        if ".csv" not in file:
            continue
        if file=="OD_baseline_Peru.csv":
            continue
        if "Partial" in file:
            continue
        scenar = read_csv(datacsv+file)
        scenar.index = scenar.Name
        n      = int(re.search('OD_scenario(.*).csv', file).group(1))
        ruc    = scenar.Total_Ta_r.sum()
        voc    = scenar.Total_Ta_v.sum()
        missingroutes = len(baseline)-len(scenar)
        baseline_nomissingroutes = baseline.ix[[i in scenar.index for i in baseline.index],:]
        diff_from_baseline = 100*(ruc/baseline_nomissingroutes.Total_Ta_r.sum()-1)
        diff_from_baseline_abs = ruc-baseline_nomissingroutes.Total_Ta_r.sum()
        diff_km_from_baseline  = scenar.Total_KM.sum()-baseline_nomissingroutes.Total_KM.sum()
        diff_km = scenar.Total_KM - baseline_nomissingroutes.Total_KM
        affected_routes = np.round(scenar.Total_Ta_r/baseline_nomissingroutes.Total_Ta_r,3)!=1
        af_cost = 100*(scenar.ix[affected_routes,"Total_Ta_r"]*baseline_nomissingroutes.ix[affected_routes,"avg_TPDA"]\
        /(baseline_nomissingroutes.ix[affected_routes,"Total_Ta_r"]*baseline_nomissingroutes.ix[affected_routes,"avg_TPDA"])-1)
        af_cost_abs = scenar.ix[affected_routes,"Total_Ta_r"]*baseline_nomissingroutes.ix[affected_routes,"avg_TPDA"]\
        -(baseline_nomissingroutes.ix[affected_routes,"Total_Ta_r"]*baseline_nomissingroutes.ix[affected_routes,"avg_TPDA"])

        costs_all.loc[len(costs_all),:] = [n,diff_from_baseline,diff_from_baseline_abs,\
                                   missingroutes,sum(affected_routes),\
                                   np.mean(af_cost_abs),np.max(af_cost),np.max(af_cost_abs),\
                                  np.mean(diff_km),np.max(diff_km),diff_km_from_baseline]

    costs_all.to_csv("costs_peru_ap22.csv",index=False)

# Second analysis: calculation of costs with weights for nodes based on the gravity model

### calculates weights for each node

In [4]:
nodes1 = read_excel(data_peru+"NODE_CID.xlsx","NODES_CID")
nodes2 = read_excel(data_peru+"NODES_10km_TPDA.xlsx","NODES_10km_TPDA")

nodes = merge(nodes1, nodes2, on='CID', how='outer')

#### the function below calculates the weight of a OD pair based on the gravity model

In [5]:
baseline["Ot"] = baseline.O.replace(nodes.set_index('CID').Sum_TPDA)
baseline["Dt"] = baseline.D.replace(nodes.set_index('CID').Sum_TPDA)
baseline["Opop"] = baseline.O.replace(nodes.set_index('CID')["Population Headcount"])
baseline["Dpop"] = baseline.D.replace(nodes.set_index('CID')["Population Headcount"])

#### here we calculate losses for each scenario

In [6]:
baseline.ix[baseline.Total_KM!=0,'weights']=baseline.ix[baseline.Total_KM!=0,'Ot']\
                                            *baseline.ix[baseline.Total_KM!=0,'Dt']\
                                            /baseline.ix[baseline.Total_KM!=0,'Total_KM']**2
baseline.weights.fillna(0,inplace=True)

In [7]:
info_links = read_excel(data_peru+"Traffic_Link_Final_PER_jun03.xlsx","Traffic_Link_Final")
#info_links = read_excel(datacsv+"allinfo.xlsx","Traffic_Link_AllInfo")

In [8]:
if os.path.exists("costs_peru_may12.csv"):
    costs_all = read_csv("costs_peru_may12.csv")
else:
    costs_all = DataFrame(columns=["scenarioID","partial_or_full","part_ruc_increase","ruc_increase","missingroutes",\
                               "num_aff_routes","cost_with_traffic","km_diff"])
    for file in list_csv:
        if ".csv" not in file:
            continue
        if file=="OD_baseline_Peru.csv":
            continue
        if ("cluster" in file):
            continue
        if ("Partial_V2" in file):
            partial_or_full="partial"
            n      = int(re.search('Scenario_OD(.*)_Partial_V2.csv', file).group(1))
            part_ruc_increase = 0.05
        elif ("Partial_v1" in file):
            partial_or_full="partial"
            n      = int(re.search('Scenario_OD(.*)_Partial_v1.csv', file).group(1))
            part_ruc_increase = 0.5
        else:
            partial_or_full='full'
            n      = int(re.search('OD_scenario(.*).csv', file).group(1))
            part_ruc_increase = 0.
        scenar = read_csv(datacsv+file)
        scenar.index = scenar.Name
        
        missingroutes = len(baseline)-len(scenar)
        # we do not take into account the routes that don't have a second best solution
        baseline_nm = baseline.ix[[i in scenar.index for i in baseline.index],:]
        # we select only routes that get affected by the disruption
        affected_routes = (np.round(scenar.Total_Ta_r/baseline_nm.Total_Ta_r,3)!=1)&(baseline_nm.Total_Ta_r>0)
        subscenar = scenar.ix[affected_routes,:]
        subscenar["weights"] = baseline_nm.weights
        
        traffic = info_links.ix[info_links.ScenarioID,"TPDA"].values[0]
            
        diff_ruc_baseline   = (subscenar.Total_Ta_r-\
                               baseline_nm.ix[affected_routes,"Total_Ta_r"])
        diff_km_from_baseline = (subscenar.Total_KM-baseline_nm.ix[affected_routes,"Total_KM"])
        diff_tot_baseline   = (traffic*diff_ruc_baseline)
        
        ruc_increase      = np.sum(diff_ruc_baseline*subscenar.weights)/subscenar.weights.sum()
        km_diff = np.sum(diff_km_from_baseline*subscenar.weights)/subscenar.weights.sum()
        cost_with_traffic  = np.sum(diff_tot_baseline*subscenar.weights)/subscenar.weights.sum()

        costs_all.loc[len(costs_all),:] = [n,partial_or_full,part_ruc_increase,\
                                           ruc_increase,missingroutes,sum(affected_routes),\
                                           cost_with_traffic,km_diff]

    costs_all.to_csv("costs_peru_may12.csv",index=False)

here I merge the info that CJ sent with the results of the analysis (calculation of disruption costs)

In [23]:
allinfo = merge(info_links, costs_all.rename(columns={"scenarioID":"ScenarioID"}), on='ScenarioID', how='inner')
allinfo.index=allinfo.ScenarioID

# Clusters of links

In [24]:
cluster1 = np.array([i in [195, 567, 568, 569, 570, 572, 719, 720] for i in allinfo.ScenarioID])
cluster2 = np.array([i in [135,136,138,139,140,545,654] for i in allinfo.ScenarioID])
cluster3 = np.array([i in [124, 126, 131, 622, 623, 624, 642, 643, 645]+list(range(594,621))\
                     for i in allinfo.ScenarioID])
cluster4 = np.array([i in [271,272] for i in allinfo.ScenarioID])
cluster5 = np.array([i in [603,604,605] for i in allinfo.ScenarioID])

In [25]:
costs_clusters = DataFrame(columns=["ScenarioID","partial_or_full",'part_ruc_increase',"ruc_increase","missingroutes",\
                           "num_aff_routes","cost_with_traffic","km_diff"])
for file in list_csv:
    if ".csv" not in file:
        continue
    if "cluster" not in file:
        continue
    if ("partial" in file):
        partial_or_full="partial"
        n      = int(re.search('OD_scenario_cluster(.*)_partial.csv', file).group(1))
        part_ruc_increase = 0.05
    else:
        partial_or_full="full"
        n      = int(re.search('OD_scenario_cluster(.*).csv', file).group(1))
        part_ruc_increase = 0.
    scenar = read_csv(datacsv+file)
    scenar.index = scenar.Name
    

    missingroutes = len(baseline)-len(scenar)
    # we do not take into account the routes that don't have a second best solution
    baseline_nm = baseline.ix[[i in scenar.index for i in baseline.index],:]
    # we select only routes that get affected by the disruption
    affected_routes = (np.round(scenar.Total_Ta_r/baseline_nm.Total_Ta_r,3)!=1)&(baseline_nm.Total_Ta_r!=0)
    subscenar = scenar.ix[affected_routes,:]
    subscenar["weights"] = baseline_nm.ix[affected_routes,'weights']

    traffic = info_links.ix[info_links.ScenarioID,"TPDA"].values[0]

    diff_ruc_baseline   = (subscenar.Total_Ta_r-\
                           baseline_nm.ix[affected_routes,"Total_Ta_r"]).dropna()
    diff_km_from_baseline = (subscenar.Total_KM-\
                             baseline_nm.ix[affected_routes,"Total_KM"]).dropna()
    diff_tot_baseline   = (traffic*diff_ruc_baseline).dropna()
    
    if sum(affected_routes)>0:
        ruc_increase      = np.sum(diff_ruc_baseline*subscenar.weights)/subscenar.weights.sum()
        km_diff = np.sum(diff_km_from_baseline*subscenar.weights)/subscenar.weights.sum()
        cost_with_traffic  = np.sum(diff_tot_baseline*subscenar.weights)/subscenar.weights.sum()
    else:
        ruc_increase      = 0
        km_diff = 0
        cost_with_traffic  = 0

    costs_clusters.loc[len(costs_clusters),:] = ["cluster{}".format(n),partial_or_full,part_ruc_increase,\
                                                 ruc_increase,missingroutes,sum(affected_routes),\
                                       cost_with_traffic,km_diff]

C:\Miniconda3\lib\site-packages\IPython\kernel\__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [26]:
costs_clusters

,ScenarioID,partial_or_full,part_ruc_increase,ruc_increase,missingroutes,num_aff_routes,cost_with_traffic,km_diff
0,cluster2,full,0,34.45108,0,412,237919.1,9.826937
1,cluster2,partial,0.05,0.5391596,0,64,3723.436,3.264278e-07
2,cluster3,full,0,122.9614,0,538,849171.4,75.89714
3,cluster3,partial,0.05,3.909676,0,538,27000.22,0.003388182
4,cluster4,full,0,449.4244,0,1874,3103725,284.8397
5,cluster4,partial,0.05,0.592524,0,70,4091.971,-8.977825e-07
6,cluster5,full,0,36.16417,0,398,249749.8,19.71234
7,cluster5,partial,0.05,0.4291071,0,34,2963.414,4.560188e-07


In [27]:
allinfo = allinfo.append(costs_clusters,ignore_index=True)

In [28]:
for clu in ['cluster2', 'cluster3', 'cluster4','cluster5']:

    for col in ['ADMIN_NAME', 'CITY_NAME', 'CLASS', 'CNTRY_NAME', 'CODIGO', 'COND1', 'CORR_ID','Identifier',\
               'LANES', 'NAME_0', 'OD', 'OPTIMAL', 'STATUS', 'STATUS.1', 'SURFACE1','TERRAIN', 'TID',]:
        allinfo.ix[allinfo.ScenarioID==clu,col]=allinfo.ix[eval(clu),col].iloc[0]

    for climat in ['EU_historical','GFDL_8.5','HadGEM2_8.5','IPSL_8.5']:
        for RP in [5,10,25,50,100,250,500,1000]:
            col = "{}_RP{} (dm)".format(climat,RP)
            allinfo.ix[allinfo.ScenarioID==clu,col]=allinfo.ix[eval(clu),col].max()

    for col in ["KM"]:
        allinfo.ix[allinfo.ScenarioID==clu,col]=allinfo.ix[eval(clu),col].sum()

    for col in ['Elevation (m)','TPDA']:
        allinfo.ix[allinfo.ScenarioID==clu,col]=allinfo.ix[eval(clu),col].max()

In [29]:
allinfo.to_csv("allinfo_peru.csv",index=False,encoding='utf-8')

In [30]:
allinfo.ix[((allinfo.ScenarioID=="cluster3")|(allinfo.ScenarioID=="cluster4")|(allinfo.ScenarioID=="cluster5"))\
           &(allinfo.partial_or_full=="full"),\
           ["ScenarioID","km_diff","ruc_increase","cost_with_traffic"]].to_csv("table1.csv")

In [36]:
allinfo.ix[(allinfo.ScenarioID=="cluster3")|(allinfo.ScenarioID=="cluster4")|(allinfo.ScenarioID=="cluster5"),"KM"]

970    84.309829
971    84.309829
972     8.401978
973     8.401978
974     7.839309
975     7.839309
Name: KM, dtype: float64

In [26]:
allinfo.columns

Index(['ADMIN_NAME', 'CITY_NAME', 'CLASS', 'CNTRY_NAME', 'CODIGO', 'COND1',
       'CORR_ID', 'EU_historical_RP10 (dm)', 'EU_historical_RP100 (dm)',
       'EU_historical_RP1000 (dm)', 'EU_historical_RP25 (dm)',
       'EU_historical_RP250 (dm)', 'EU_historical_RP5 (dm)',
       'EU_historical_RP50 (dm)', 'EU_historical_RP500 (dm)', 'Elevation (m)',
       'GFDL_8.5_RP10 (dm)', 'GFDL_8.5_RP100 (dm)', 'GFDL_8.5_RP1000 (dm)',
       'GFDL_8.5_RP25 (dm)', 'GFDL_8.5_RP250 (dm)', 'GFDL_8.5_RP5 (dm)',
       'GFDL_8.5_RP50 (dm)', 'GFDL_8.5_RP500 (dm)', 'HadGEM2_8.5_RP10 (dm)',
       'HadGEM2_8.5_RP100 (dm)', 'HadGEM2_8.5_RP1000 (dm)',
       'HadGEM2_8.5_RP25 (dm)', 'HadGEM2_8.5_RP250 (dm)',
       'HadGEM2_8.5_RP5 (dm)', 'HadGEM2_8.5_RP50 (dm)',
       'HadGEM2_8.5_RP500 (dm)', 'INTERVEN', 'IPSL_8.5_RP10 (dm)',
       'IPSL_8.5_RP100 (dm)', 'IPSL_8.5_RP1000 (dm)', 'IPSL_8.5_RP25 (dm)',
       'IPSL_8.5_RP250 (dm)', 'IPSL_8.5_RP5 (dm)', 'IPSL_8.5_RP50 (dm)',
       'IPSL_8.5_RP500 (dm)', 'I

In [27]:
allinfo.ix[cluster4,["CLASS","COND1","KM","LANES","SURFACE1","TERRAIN"]]

,CLASS,COND1,KM,LANES,SURFACE1,TERRAIN
273,Primary,Poor,6.246163,6,Paved,Mostly straight and gently undulating
274,Primary,Poor,2.155815,6,Paved,Mostly straight and gently undulating


In [28]:
allinfo.ix[cluster3,["CLASS","COND1","KM","LANES","SURFACE1","TERRAIN"]]

,CLASS,COND1,KM,LANES,SURFACE1,TERRAIN
124,Primary,Good,9.596965,6,Paved,Bendy and severely undulating
125,Primary,Good,9.596965,6,Paved,Bendy and severely undulating
126,Primary,Good,9.596965,6,Paved,Bendy and severely undulating
128,Primary,Good,1.107098,6,Paved,Bendy and severely undulating
133,Primary,Good,1.420506,6,Paved,Bendy and severely undulating
600,Primary,Good,1.365934,6,Paved,Bendy and severely undulating
601,Primary,Good,2.481374,6,Paved,Bendy and severely undulating
602,Primary,Good,3.503798,6,Paved,Bendy and severely undulating
603,Primary,Good,0.506373,6,Paved,Mostly straight and gently undulating
604,Primary,Good,0.107644,6,Paved,Mostly straight and gently undulating


In [17]:
allinfo.ix[cluster5,["CLASS","COND1","KM","LANES","SURFACE1","TERRAIN"]]

,CLASS,COND1,KM,LANES,SURFACE1,TERRAIN
609,Primary,Good,3.210257,6,Paved,Mostly straight and gently undulating
610,Primary,Good,2.537947,6,Paved,Mostly straight and gently undulating
611,Primary,Good,2.091105,6,Paved,Mostly straight and gently undulating
